In [38]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pandas as pd

In [54]:
n_totals = [5000, 10000, 15000, 20000, 25000, 30000, 35000]
delta_bs = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6]

all_permutations = [
    i
    for i in it.product(
        n_totals,
        delta_bs,
    )
]
all_permutations = np.array(all_permutations)

# parameter vectors to run through with cl_idx
cl_n_totals = all_permutations[:, 0].astype(int)
cl_delta_bs = all_permutations[:, 1]

def get_string_data(cl_idx):
    save_str = (
            "results/resolution/constant_idx/"
            "p_val" + str(cl_idx) + ".csv"
        )
    return save_str


def get_df(cl_idx):
    all_pvals = []
    all_delta_bs = []
    all_n_totals = []

    for cl_idx in range(len(cl_n_totals)):
        cl_idx = 4
        save_str = get_string_data(cl_idx)
        try :
            pvals = np.genfromtxt('../'+save_str, delimiter=',')
        except :
            pvals = []

        all_pvals.extend(pvals)
        all_delta_bs.extend([cl_delta_bs[cl_idx]]*len(pvals))
        all_n_totals.extend([cl_n_totals[cl_idx]]*len(pvals))

    # make a dataframe
    df = pd.DataFrame({
        "pval": all_pvals,
        "delta b": all_delta_bs,
        "n_total": all_n_totals,
    })
    return df

In [55]:
df = get_df(0)

In [56]:
df

,pval,delta b,n_total
0,0.003176,0.3,5000
1,0.001506,0.3,5000
2,0.021916,0.3,5000
3,0.187602,0.3,5000
4,0.000737,0.3,5000
...,...,...,...
995,0.234099,0.3,5000
996,0.059567,0.3,5000
997,0.338979,0.3,5000
998,0.247243,0.3,5000
